# Projeto 1 - Ciência dos Dados

Nome: André Faia Vasconcelos Ramos

Nome: Enzo Fonteyne 

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
#Importação das bibliotecas que serão utilizadas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\andre


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Excel-Samsung.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Nota
0,@devilslawyerr_ toda vez q eu entro no tiktok ...,1
1,@amarcosta01 @qgjulietteof @seara_brasil tb am...,0
2,@prodzefyre @luagluiz @brrrb3rry um samsung e ...,3
3,é humanamente impossível comprar uma máquina l...,2
4,[magazine luiza] (cliente ouro) carregador por...,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Nota
0,"@ai_cansado esmola da samsung, ainda não tô pr...",0.0
1,vsf essa propaganda da samsung com a juliettr\...,0.0
2,o bts ainda fez uma propaganda pra samsung no ...,0.0
3,aaaaaaaaaaaaa vem aí????? não brinca com meu j...,0.0
4,"eu defendia a samsung, mas agr a juliette é ga...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


0 - Irrelevante - Os tweets considerados irrelevantes são aqueles que não fazem menção aos produtos da Samsung

1 - Neutro - Os tweets considerados neutros são aqueles que fazem uma menção a algum produto da Samsung, porém não é algo significativo. Ex: aaa samsung eu to travada que nem o celular de vcs

2 - Relevante - Os tweets considerados relevantes são aqueles que fazem uma menção construtiva a algum produto da Samsung ou alguma funcionalidade que eles apresentam

3 - Muito Relevante - Os tweets considerados muito relevantes são aqueles que fazem uma menção construtiva de 2 ou + produtos da Samsumg ou 2 ou + funcionalidades que eles apresentam. Isso inclui venda de produtos com descrições detalhadas


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#Removendo pontuações
def limpeza_caracteres_iniciais(tweet):
    caracteres_especiais = '[!-.:?;_,<>/\][)(#%$|\n]' 
    pattern = re.compile(caracteres_especiais)
    substituições = re.sub(pattern,'', tweet)
    return substituições

#Removendo menções
def limpeza_menções(tweet):
    mencoes = "@[A-Za-z0-9]+"
    pattern = re.compile(mencoes)
    substituições = re.sub(pattern,'',tweet)
    return substituições

train.Treinamento = train.Treinamento.apply(limpeza_caracteres_iniciais)
train.Treinamento = train.Treinamento.apply(limpeza_menções)

train

,Treinamento,Nota
0,toda vez q eu entro no tiktok ta essa p no an...,1
1,tb amo miga 😍🎵💜📱mpn juliettefashion juliet...,0
2,um samsung e um xiaomi de 500 conto não val...,3
3,é humanamente impossível comprar uma máquina l...,2
4,magazine luiza cliente ouro carregador portáti...,1
...,...,...
695,nss a juli finalmente anunciando a parceria de...,0
696,samsung pq demorou tanto espero essa parceri...,0
697,amg se for pouco demais tu leva numa assistê...,1
698,não brinca com isso samsung,0


In [7]:
#Separando os tweets
irrelevante = train.loc[train.Nota == 0, :]
neutro = train.loc[train.Nota == 1, :]
relevante = train.loc[train.Nota == 2, :]
muito_relevante = train.loc[train.Nota == 3, :]

In [8]:
irrelevante

,Treinamento,Nota
1,tb amo miga 😍🎵💜📱mpn juliettefashion juliet...,0
8,adoraria que o galiotte pegasse os 300 milh...,0
12,✅curte ✅comenta ✅compatilhar✅salve a foto da ...,0
14,hoje eu acordei daquele jeitinho do coringa so...,0
16,httpstcooigc3turnh hj foi tão bom space da ju ...,0
...,...,...
693,oi nesse link eu trago procedimentos para a o...,0
695,nss a juli finalmente anunciando a parceria de...,0
696,samsung pq demorou tanto espero essa parceri...,0
698,não brinca com isso samsung,0


In [9]:
relevante

,Treinamento,Nota
3,é humanamente impossível comprar uma máquina l...,2
6,pucrio apple e samsung abrem capacitação em te...,2
11,nunca falta la promo a samsung lmao,2
13,samsung segue xiaomi e lança ram virtual em ce...,2
17,compre online eletrônicos e tecnologia encontr...,2
...,...,...
669,adicionar relogio na tela inicial do celular s...,2
678,submarino preção😱‼fone de ouvido samsung galax...,2
686,menino tu me deu uma ideia agora vo baixar no...,2
688,nossa linha j da samsung é um terror kkkkkobr...,2


In [10]:
muito_relevante

,Treinamento,Nota
2,um samsung e um xiaomi de 500 conto não val...,3
9,quero eu um samsung desse pow,3
22,essa cam da samsung um lixo dois pixels tá ótimo,3
33,samsung é um péssimo atendimento só indo pra j...,3
34,eu sinceramente quando joguei no seu cll ac...,3
...,...,...
655,samsung até no espaço meus amigos,3
660,alguém tá tendo dificuldade em reproduzir con...,3
661,magalu está faltando opções de smartwatchs no...,3
676,smartphone samsung galaxy s20 fe 128gb cloud n...,3


In [11]:
#Fazendo a contagem das palavras em cada classificação dos tweets
for i,tweet in enumerate(irrelevante.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_irrelevante = tweet
    else:
        lista_irrelevante += tweet
    
Irrelevante = pd.Series(lista_irrelevante)
Irrelevante.value_counts()

samsung                 230
a                       131
o                       122
de                      120
da                      120
                       ... 
coldplay                  1
email                     1
loccitanealimentação      1
simplesmente              1
tottenham                 1
Length: 1508, dtype: int64

In [12]:
#Fazendo a contagem das palavras em cada classificação dos tweets
for i,tweet in enumerate(relevante.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_relevante = tweet
    else:
        lista_relevante += tweet
    
Relevante = pd.Series(lista_relevante)
Relevante.value_counts()

samsung     147
de           81
que          79
o            76
e            76
           ... 
a30           1
olhada        1
continuo      1
ganhei        1
chip          1
Length: 1166, dtype: int64

In [13]:
#Fazendo a contagem das palavras em cada classificação dos tweets
for i,tweet in enumerate(muito_relevante.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_muito_relevante = tweet
    else:
        lista_muito_relevante += tweet
    
Muito_Relevante = pd.Series(lista_muito_relevante)
Muito_Relevante.value_counts()

samsung        106
e               66
de              58
da              50
a               47
              ... 
week             1
64gb             1
fabrica          1
popularizar      1
tentar           1
Length: 1061, dtype: int64

In [14]:
#Juntando todas as palavras e fazendo a contagem delas
palavras = lista_relevante + lista_irrelevante + lista_muito_relevante
serie_palavras = pd.Series(palavras)
serie_palavras.value_counts().head(50)

samsung        483
de             259
a              253
e              235
o              229
da             226
que            204
eu             149
é              141
um             139
com            136
no             112
não             93
do              90
pra             72
na              67
meu             63
mais            60
celular         59
me              58
q               57
uma             54
em              51
por             50
se              48
juliette        48
tem             48
tiktok          47
os              45
mas             45
só              43
propaganda      40
para            35
essa            35
vai             34
as              32
tá              32
galaxy          30
iphone          29
pq              29
despertador     28
já              27
vou             27
esse            27
muito           25
isso            24
ou              22
agora           21
ela             21
ele             21
dtype: int64

In [15]:
#Eliminando palavras irrelevantes para a classificação
stopwords = ['a','de','e','o','da','que','com','eu','é','um','no','não','do','na','pra','q','me','uma','em','por']
for palavra in lista_relevante:
    if palavra in stopwords:
        lista_relevante.remove(palavra)
for palavra in lista_irrelevante:
    if palavra in stopwords:
        lista_irrelevante.remove(palavra)
for palavra in lista_muito_relevante:
    if palavra in stopwords:
        lista_muito_relevante.remove(palavra)


In [16]:
#Transformação para pd series e cálculo relativo
serie_relevante = pd.Series(lista_relevante)
serie_irrelevante = pd.Series(lista_irrelevante)
serie_muito_relevante = pd.Series(lista_muito_relevante)

tabela_relevante = serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_irrelevante = serie_irrelevante.value_counts()
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_muito_relevante = serie_muito_relevante.value_counts()
tabela_muito_relevante_relativa = serie_muito_relevante.value_counts(True)

In [17]:
#Cálculo de probabilidades individuais
probRel = pd.Series(tabela_relevante_relativa)
probIrel = pd.Series(tabela_irrelevante_relativa)
probMtorel = pd.Series(tabela_muito_relevante_relativa)

In [18]:
#Criação de uma lista geral de palavras
palavras = lista_relevante + lista_irrelevante + lista_muito_relevante

serie_palavras = pd.Series(palavras)
tabela_palavras = serie_palavras.value_counts()
tabela_palavras_relativa = serie_palavras.value_counts(True)

In [19]:
rel = tabela_relevante.sum()/tabela_palavras.sum()
irrel = tabela_irrelevante.sum()/tabela_palavras.sum()
mr = tabela_muito_relevante.sum()/tabela_palavras.sum()

In [20]:
tweet = "@fulano o samsung é muito bom! acho que os celulares são muito melhores do que os da apple"

In [21]:
tweet.split()

['@fulano',
 'o',
 'samsung',
 'é',
 'muito',
 'bom!',
 'acho',
 'que',
 'os',
 'celulares',
 'são',
 'muito',
 'melhores',
 'do',
 'que',
 'os',
 'da',
 'apple']

In [22]:
#Separação em value counts
tabela_relevante = Relevante.value_counts()
tabela_muito_relevante = Muito_Relevante.value_counts()
tabela_irelevante = Irrelevante.value_counts()

In [23]:
#Fazendo a função comparadora.
def comparar(texto):
    #Limpando o tweet e separando as palavras individualmente
    texto_limpo = limpeza_menções(texto)
    texto_limpo = limpeza_caracteres_iniciais(texto_limpo)
    lista_texto = texto_limpo.split()
            
    P_texto_dado_rel = 1
    P_texto_dado_irrel = 1
    P_texto_dado_mr = 1
    for palavra in lista_texto:
        if palavra not in tabela_relevante:
            P_texto_dado_rel = P_texto_dado_rel/(tabela_relevante.sum() + len(lista_texto))
        else:
            P_texto_dado_rel = P_texto_dado_rel*(tabela_relevante[palavra] + 1)/(tabela_relevante.sum() + len(lista_texto))
        if palavra not in tabela_irrelevante:
            P_texto_dado_irrel = P_texto_dado_irrel/(tabela_irrelevante.sum() + len(lista_texto))
        else:
            P_texto_dado_irrel = P_texto_dado_irrel*(tabela_irrelevante[palavra] + 1)/(tabela_irrelevante.sum() + len(lista_texto))
        if palavra not in tabela_muito_relevante:
            P_texto_dado_mr = P_texto_dado_irrel/(tabela_muito_relevante.sum() + len(lista_texto))
        else:
            P_texto_dado_mr = P_texto_dado_irrel*(tabela_muito_relevante[palavra] + 1)/(tabela_muito_relevante.sum() + len(lista_texto))
    
    print(P_texto_dado_rel)
    #Calculando as probabilidades 
    P_rel_dado_texto = P_texto_dado_rel * rel
    P_irrel_dado_texto = P_texto_dado_irrel * irrel
    P_mr_dado_texto = P_texto_dado_irrel * mr
    #Retornando a classificação do tweet
    if P_rel_dado_texto > P_irrel_dado_texto and P_rel_dado_texto > P_mr_dado_texto:
        return "Relevante"
    if P_irrel_dado_texto > P_rel_dado_texto and P_irrel_dado_texto > P_mr_dado_texto:
        return "Irrelevante"
    if P_mr_dado_texto > P_irrel_dado_texto and P_mr_dado_texto > P_rel_dado_texto:
        return "Muito relevante"

In [24]:
comparar(tweet)

5.5922754985564264e-39


'Relevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [25]:
test

,Teste,Nota
0,"@ai_cansado esmola da samsung, ainda não tô pr...",0.0
1,vsf essa propaganda da samsung com a juliettr\...,0.0
2,o bts ainda fez uma propaganda pra samsung no ...,0.0
3,aaaaaaaaaaaaa vem aí????? não brinca com meu j...,0.0
4,"eu defendia a samsung, mas agr a juliette é ga...",0.0
...,...,...
295,@mn_rcfx @djtheusdags imagina o psicólogico do...,1.0
296,mais de um mês sem acesso ao meu aplicativo de...,0.0
297,confira o vídeo de samsung brasil! #tiktok htt...,1.0
298,"hoje é sexta, dia de travar mais que a samsung!",0.0


In [26]:
test = pd.read_excel(filename, sheet_name = 'Teste')
tweets = test.loc[:,'Teste']

test['Nota'] = test['Nota'].astype('category')
test['Nota'].cat.categories = ['Irrelevante', 'Relevante','Neutro','Muito relevante']
test.rename(columns={"Nota":"Real"}, inplace = "True")

In [27]:
#Classificação numérica dos tweets
tweets_classificados = []
for linha in tweets:
    tweets_classificados.append(comparar(linha))

2.17783375253927e-32
5.247738854137248e-54
4.8815253837796425e-28
3.246952346547551e-31
1.6915561183811483e-53
1.5809481363360165e-79
1.5181349227609488e-29
8.199226110662366e-35
3.7049624174063065e-71
8.083969669172195e-156
3.506660723328064e-21
8.644661538597069e-114
5.761883208976656e-33
1.0264962084686139e-25
5.336741610902522e-121
3.1834707662993056e-44
2.1966963913664214e-104
5.5775010770172115e-33
2.327796067439442e-136
3.7547632320181667e-53
9.84300621464818e-83
2.5776161877460555e-64
2.74344633060372e-20
3.5987784206575945e-20
6.580066191462405e-36
1.3920707209266882e-21
2.0598230565888665e-31
2.518730903892365e-40
9.365501224852002e-83
3.989581579650367e-62
2.0587792932626067e-113
1.1178045336007488e-10
4.786130484616053e-20
1.1099805283381883e-163
1.275791678879627e-62
5.336741610902522e-121
8.720859870185387e-56
7.017854516796703e-14
6.735151926614715e-38
2.0528979744949127e-62
9.871046238628985e-34
4.782337220219853e-72
4.246166821638044e-07
3.945070768789414e-48
4.0417718

In [28]:
test['Bayes'] = pd.Series(tweets_classificados)

In [29]:
test

,Teste,Real,Bayes
0,"@ai_cansado esmola da samsung, ainda não tô pr...",Irrelevante,Relevante
1,vsf essa propaganda da samsung com a juliettr\...,Irrelevante,Relevante
2,o bts ainda fez uma propaganda pra samsung no ...,Irrelevante,Relevante
3,aaaaaaaaaaaaa vem aí????? não brinca com meu j...,Irrelevante,Irrelevante
4,"eu defendia a samsung, mas agr a juliette é ga...",Irrelevante,Relevante
...,...,...,...
295,@mn_rcfx @djtheusdags imagina o psicólogico do...,Relevante,Relevante
296,mais de um mês sem acesso ao meu aplicativo de...,Irrelevante,Relevante
297,confira o vídeo de samsung brasil! #tiktok htt...,Relevante,Irrelevante
298,"hoje é sexta, dia de travar mais que a samsung!",Irrelevante,Relevante


In [38]:
#Verificando taxa de verdadeiros positivos
tab_Vpos = test.loc[(test.Real=='Relevante') & (test.Bayes=='Relevante') ]
Vpos = 100*len(tab_Vpos)/len(test)
#Verificando taxa de verdadeiros negativos
tab_Vneg = test.loc[(test.Real=='Irrelevante') & (test.Bayes=='Irrelevante') ]
Vneg = 100*len(tab_Vneg)/len(test)
#Verificando taxa de falsos positivos
tab_Fpos = test.loc[(test.Real=='Irrelevante') & (test.Bayes=='Relevante') ]
Fpos = 100*len(tab_Fpos)/len(test)
#Verificando taxa de falsos negativos
tab_Fneg = test.loc[(test.Real=='Relevante') & (test.Bayes=='Irrelevante') ]
Fneg = 100*len(tab_Fneg)/len(test)
#Verificando probabilidade de tweets relevantes reais
relevante = test.loc[(test.Real=='Relevante')]
prob_rel = 100*len(relevante)/len(test)
prob_irrel = 100 - prob_rel
#Verificando probabilidade de tweets relevantes do classificador NB
relevante_b = test.loc[(test.Bayes=='Relevante')]
prob_rel_b = 100*len(relevante_b)/len(test)
prob_irrel_b = 100 - prob_irrel

In [39]:
Vpos + Vneg + Fpos + Fneg

73.66666666666666

In [32]:
print(f'A taxa de verdadeiros positivos foi {Vpos}%')
print(f'A taxa de verdadeiros negativos foi {Vneg}%')
print(f'A taxa de falsos positivos foi {Fpos}%')
print(f'A taxa de falsos negativos foi {Fneg}%')

A taxa de verdadeiros positivos foi 26.0%
A taxa de verdadeiros negativos foi 4.666666666666667%
A taxa de falsos positivos foi 38.666666666666664%
A taxa de falsos negativos foi 4.333333333333333%


In [33]:
print(f'A taxa de tweets classificados corretamente foi {Vpos+Vneg}%')
print(f'A taxa de tweets classificados incorretamente foi {Fpos+Fneg}%')

A taxa de tweets classificados corretamente foi 30.666666666666668%
A taxa de tweets classificados incorretamente foi 43.0%


In [34]:
print(f'A taxa de tweets classificados incorretamente na parte relevante era {Fpos+Vpos/prob_rel_b}%')

A taxa de tweets classificados incorretamente na parte relevante era 38.95238095238095%


___
### Concluindo

**Concluímos que o classificador apesar de não ter tido uma eficácia tão alta, ele funciona corretamente. Para melhorar o seu rendimento é nescessário mais precisão na classificação manual, podemos dizer que a imprecisão do ser humano atrapalhou e confundiu o classificador e abaixou a sua eficácia.**

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

**O classificador em si é funcional a partir do momento em que lhe são entregues valores com precisões muito elevadas, sendo assim havendo a separação se permite ensinar ao classificador o que deve caber em cada classificação, porém ao mesmo tempo, a classificação ocorre de forma manual e como consequência possui uma taxa de erro razoavelmente elevada, causando um ensinamento errôneo ao classificador**

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**